In [29]:
import numpy as np
import pandas as pd
from collections import defaultdict
import operator, os
import time, datetime
from utils import *
from tqdm import tqdm

# 1. statistics overview

In [37]:
bath_path = '/mnt/qb/work/mlcolab/hzhou52/kt/assistment17'
file_name = 'anonymized_full_release_competition_dataset.csv'
log = pd.read_csv(os.path.join(bath_path, file_name), encoding = "utf-8",low_memory=False)

In [3]:
print('The columns of log are ', log.columns)
print('The length of log is {}'.format(len(log)))

print('###########################################')

print(count_unique(log, log.columns))

The columns of log are  Index(['studentId', 'MiddleSchoolId', 'InferredGender', 'SY ASSISTments Usage',
       'AveKnow', 'AveCarelessness', 'AveCorrect', 'NumActions', 'AveResBored',
       'AveResEngcon', 'AveResConf', 'AveResFrust', 'AveResOfftask',
       'AveResGaming', 'action_num', 'skill', 'problemId', 'problemType',
       'assignmentId', 'assistmentId', 'startTime', 'endTime', 'timeTaken',
       'correct', 'original', 'hint', 'hintCount', 'hintTotal', 'scaffold',
       'bottomHint', 'attemptCount', 'frIsHelpRequest', 'frPast5HelpRequest',
       'frPast8HelpRequest', 'stlHintUsed', 'past8BottomOut',
       'totalFrPercentPastWrong', 'totalFrPastWrongCount', 'frPast5WrongCount',
       'frPast8WrongCount', 'totalFrTimeOnSkill', 'timeSinceSkill',
       'frWorkingInSchool', 'totalFrAttempted', 'totalFrSkillOpportunities',
       'responseIsFillIn', 'responseIsChosen', 'endsWithScaffolding',
       'endsWithAutoScaffolding', 'frTimeTakenOnScaffolding',
       'frTotalSkillOppo

In [4]:
for col in log.columns:
    print(col, log[col][0:5])

studentId 0    8
1    8
2    8
3    8
4    8
Name: studentId, dtype: int64
MiddleSchoolId 0    2
1    2
2    2
3    2
4    2
Name: MiddleSchoolId, dtype: int64
InferredGender 0    Male
1    Male
2    Male
3    Male
4    Male
Name: InferredGender, dtype: object
SY ASSISTments Usage 0    2004-2005
1    2004-2005
2    2004-2005
3    2004-2005
4    2004-2005
Name: SY ASSISTments Usage, dtype: object
AveKnow 0    0.352416
1    0.352416
2    0.352416
3    0.352416
4    0.352416
Name: AveKnow, dtype: float64
AveCarelessness 0    0.183276
1    0.183276
2    0.183276
3    0.183276
4    0.183276
Name: AveCarelessness, dtype: float64
AveCorrect 0    0.483902
1    0.483902
2    0.483902
3    0.483902
4    0.483902
Name: AveCorrect, dtype: float64
NumActions 0    1056
1    1056
2    1056
3    1056
4    1056
Name: NumActions, dtype: int64
AveResBored 0    0.208389
1    0.208389
2    0.208389
3    0.208389
4    0.208389
Name: AveResBored, dtype: float64
AveResEngcon 0    0.679126
1    0.679126
2    0

In [38]:
interested_col = [
    'studentId', 'problemId', 'skill', 'startTime', 'endTime', 'correct'
]
invert_col = [
    'user_id', 'problem_id', 'skill_text', 'timestamp', 'dwell_time', 'correct', 
]

In [6]:
check_nan(log)
count_unique(log, ['studentId', 'skill', 'problemId'])
print(len(log))

Number of NaN values in column studentId: 0
Number of NaN values in column MiddleSchoolId: 0
Number of NaN values in column InferredGender: 173656
Number of NaN values in column SY ASSISTments Usage: 0
Number of NaN values in column AveKnow: 0
Number of NaN values in column AveCarelessness: 0
Number of NaN values in column AveCorrect: 0
Number of NaN values in column NumActions: 0
Number of NaN values in column AveResBored: 0
Number of NaN values in column AveResEngcon: 0
Number of NaN values in column AveResConf: 0
Number of NaN values in column AveResFrust: 0
Number of NaN values in column AveResOfftask: 0
Number of NaN values in column AveResGaming: 0
Number of NaN values in column action_num: 0
Number of NaN values in column skill: 0
Number of NaN values in column problemId: 0
Number of NaN values in column problemType: 0
Number of NaN values in column assignmentId: 0
Number of NaN values in column assistmentId: 0
Number of NaN values in column startTime: 0
Number of NaN values in 

In [8]:
log['correct'].mean()

0.37268141397685234

# 2. remove learner logs with threshold

In [13]:
remove_thres = [50, 100, 150, 200]

for thres in remove_thres:
    print('remove threshold is {}'.format(thres))
    log = log[~log['skill'].isna()]
    log = remove_log(log, 'studentId', num=thres)
    count_unique(log, ['studentId', 'skill', 'problemId'])
    print(len(log))

remove threshold is 50
Number of unique values in studentId: 1697
Number of unique values in skill: 102
Number of unique values in problemId: 3162
942489
remove threshold is 100
Number of unique values in studentId: 1649
Number of unique values in skill: 102
Number of unique values in problemId: 3162
938790
remove threshold is 150
Number of unique values in studentId: 1512
Number of unique values in skill: 102
Number of unique values in problemId: 3158
921562
remove threshold is 200
Number of unique values in studentId: 1394
Number of unique values in skill: 102
Number of unique values in problemId: 3142
900826


In [39]:
log['startTime']

0         1096470301
1         1096470350
2         1096470354
3         1096470360
4         1096470378
             ...    
942811    1147447464
942812    1147447468
942813    1147447539
942814    1147447543
942815    1147447546
Name: startTime, Length: 942816, dtype: int64

In [42]:
remove_thres = [50, 100, 150, 200]
user_index = 'studentId'

bath_path = '/mnt/qb/work/mlcolab/hzhou52/kt/assistment17'
file_name = 'anonymized_full_release_competition_dataset.csv'
log = pd.read_csv(os.path.join(bath_path, file_name), encoding = "utf-8",low_memory=False)

for thres in remove_thres:
    print(f'Remove threshold is {thres}')

    # Remove rows with NaN skill_id and users who appear less than thres times
    log = log[~log.skill.isna()]
    log = remove_log(log, user_index, num=thres)

    # Create new DataFrame with desired columns
    df = log[interested_col].copy()
    df.columns = invert_col

    # Convert timestamp columns to a single timestamp value
    start_times = log.startTime
    end_times = log.endTime
    df['timestamp'] = (start_times + end_times) / 2
    df['dwell_time'] = end_times - start_times

    # Re-index columns
    df['skill_id'] = pd.Categorical(df['skill_text'], categories=df['skill_text'].unique()).codes
    df['problem_id'] = pd.Categorical(df['problem_id'], categories=df['problem_id'].unique()).codes
    df['user_id'] = pd.Categorical(df['user_id'], categories=df['user_id'].unique()).codes
    df = df.astype({'timestamp': np.float64, 'dwell_time': np.float64, 'correct': np.float64})
    df = df.astype({'problem_id': np.int64, 'user_id': np.int64, 'skill_id': np.int64})
    
    # Save cleaned DataFrame to a CSV file
    df.to_csv(f'/mnt/qb/work/mlcolab/hzhou52/kt/assistment17/multi_skill/interactions_{thres}.csv', sep='\t', index=False)

    print(f'Remove threshold {thres} is done!')
    print(count_unique(df, df.columns.tolist()))
    print(f'Length of DataFrame: {len(df)}')

Remove threshold is 50
Remove threshold 50 is done!
Number of unique values in user_id: 1697
Number of unique values in problem_id: 3162
Number of unique values in skill_text: 102
Number of unique values in timestamp: 807261
Number of unique values in dwell_time: 1460
Number of unique values in correct: 2
Number of unique values in skill_id: 102
None
Length of DataFrame: 942489
Remove threshold is 100
Remove threshold 100 is done!
Number of unique values in user_id: 1649
Number of unique values in problem_id: 3162
Number of unique values in skill_text: 102
Number of unique values in timestamp: 804175
Number of unique values in dwell_time: 1441
Number of unique values in correct: 2
Number of unique values in skill_id: 102
None
Length of DataFrame: 938790
Remove threshold is 150
Remove threshold 150 is done!
Number of unique values in user_id: 1512
Number of unique values in problem_id: 3158
Number of unique values in skill_text: 102
Number of unique values in timestamp: 789931
Number of

# 3. extract single learner 

In [43]:
remove_thres = [50, 100, 150, 200]
user_index = 'studentId'
skill_index = 'skill'

bath_path = '/mnt/qb/work/mlcolab/hzhou52/kt/assistment17'
file_name = 'anonymized_full_release_competition_dataset.csv'
log = pd.read_csv(os.path.join(bath_path, file_name), encoding = "utf-8",low_memory=False)

users = []
users_log = log.groupby([user_index, skill_index]).size().reset_index(name='count')

for thres in remove_thres:
    users.append(users_log[users_log['count']>thres])

In [44]:
users[0]

,studentId,skill,count
19,8,equation-solving,55
36,8,mean,66
70,8,symbolization-articulation,51
102,9,noskill,58
157,11,pattern-finding,56
...,...,...,...
71078,7775,p-patterns-relations-algebra,51
71082,7775,probability,106
71113,7778,noskill,115
71152,7782,probability,84


In [45]:
######################################## single_user_single_exercise
useful_log = log[interested_col]
remove_thres = [50, 100, 150, 200]

users_logs_threshold = []
for i in range(len(remove_thres)):
    thres = remove_thres[i]
    users_with_threshold = users[i]
    
    single_skill = {col: [] for col in interested_col}

    sk_dfs = []
    print('With threshold {} there are {} logs'.format(thres, len(users_with_threshold)))
    for j in tqdm(range(len(users_with_threshold))):
        id = users_with_threshold[user_index].iloc[j]
        ex = users_with_threshold[skill_index].iloc[j]

        single_user_exercise = useful_log.loc[(useful_log[user_index] == id) & (useful_log[skill_index] == ex)]
        single_user_exercise = single_user_exercise.sort_values('startTime')[:thres]

        sk_dfs.append(single_user_exercise)
    users_logs_threshold.append(sk_dfs)

With threshold 50 there are 3268 logs


100%|██████████| 3268/3268 [02:24<00:00, 22.62it/s]


With threshold 100 there are 699 logs


100%|██████████| 699/699 [00:30<00:00, 22.76it/s]


With threshold 150 there are 200 logs


100%|██████████| 200/200 [00:08<00:00, 22.81it/s]


With threshold 200 there are 55 logs


100%|██████████| 55/55 [00:02<00:00, 22.84it/s]


In [46]:
invert_col

['user_id', 'problem_id', 'skill_text', 'timestamp', 'dwell_time', 'correct']

In [47]:
for i in range(len(users_logs_threshold)):
    test = users_logs_threshold[i]
    test = pd.concat(test, axis=0)
    thres = remove_thres[i]

    test.columns = invert_col
    new_id = [[i] * thres for i in range(len(users_logs_threshold[i]))]
    new_user_id = [item for sublist in new_id for item in sublist]
    test['user_id'] = new_user_id

    test['skill_id'] = pd.Categorical(test['skill_text'], categories=test['skill_text'].unique()).codes
    test['problem_id'] = pd.Categorical(test['problem_id'], categories=test['problem_id'].unique()).codes

    # Convert timestamp columns to a single timestamp value
    start_times = test.timestamp
    end_times = test.dwell_time
    test['timestamp'] = (start_times + end_times) / 2
    test['dwell_time'] = end_times - start_times

    # Re-index columns
    test = test.astype({'timestamp': np.float64, 'dwell_time': np.float64, 'correct': np.float64})
    test = test.astype({'problem_id': np.int64, 'user_id': np.int64, 'skill_id': np.int64})

    # Save
    bath_path = '/mnt/qb/work/mlcolab/hzhou52/kt/assistment17/single_skill/'
    test.to_csv(bath_path+'interactions_{}.csv'.format(thres), sep='\t', index=False)

In [48]:
log['startTime']

0         1096470301
1         1096470350
2         1096470354
3         1096470360
4         1096470378
             ...    
942811    1147447464
942812    1147447468
942813    1147447539
942814    1147447543
942815    1147447546
Name: startTime, Length: 942816, dtype: int64